# Placement

This is based on the work of "Architecture-aware FPGA placement using metric embedding"
The objective for a metric based placement will be: $||D_DP-PD_A||_F^2$ where P is the permutation matrix and D_x is the distance matrix.
The constraints on permutation matrix are: $\sum_{i=1}^n P(i,j) = 1, \forall j$ 

and $\sum_{j=1}^n P(i,j) = 1, \forall i$

Casting the placement into QUBO:

-objective
$f=\sum_I^n\sum_J^n\{-2D_{II}A_{JJ}x_{IJ}+\sum_i^n[D_{Ii}^2x_{iJ}+A_{iJ}^2x_{Ii}+\sum_{j\neq i}^n (D_{Ii}D_{Ij}x_{iJ}x_{jJ}+A_{iJ}A_{jJ}x_{Ii}x_{Ij})]-\sum_{i\neq I}^n\sum_{j\neq J}^n[A_{jJ}D_{Ii}x_{iJ}x_{Ij}]-\sum_{i\neq J}^n\sum_{j\neq I}^n[A_{iJ}D_{Ij}x_{Ii}x_{jJ}]\}$

-constraint on row and columns separately (in case we need to relax one):
$\sum_j^n [1-\sum_i^nx_{ij}]^2=\sum_j^n\sum_i^n[-x_{ij}+2\sum_{k\neq i}^nx_{ij}x_{kj}]$

$\sum_i^n [1-\sum_j^nx_{ij}]^2=\sum_i^n\sum_j^n[-x_{ij}+2\sum_{k\neq j}^nx_{ij}x_{ik}]$

In [ ]:
#distance matrix calculator

In [6]:
from essentials import *
from itertools import product as prd

In [23]:
n=4

In [24]:
def set_qubo_val(Q, x1, x2, value):
    if (x1, x2) in Q.keys():
        Q[(x1, x2)] += value
    elif (x2, x1) in Q.keys():
        Q[(x2, x1)] += value
    else:
        raise Exception('Your key is not in the Q dict.') 
        
def placement_qubo(D_design, D_architecture, params={'weight_objective': 1, 'weight_row': 1, 'weight_column': 1}):
    #D_design and D_architecture matrices are 2-D numpy arrays
    Q = {}
    n = len(D_design)
    # for now the design and architecture distance matrices are of the same size
    
    permutation_matrix_elements=[]
    for var1, var2 in prd(range(0, n), repeat=2):
        permutation_matrix_elements.append([var1, var2])
        
    for var1, var2 in cwr(permutation_matrix_elements, 2):
            Q[(f'x{var1[0]}{var1[1]}', f'x{var2[0]}{var2[1]}')] = 0
    
    # Objective
    w1 = params['weight_objective']
    for I, J in prd(range(0,n), repeat=2):
        Q[(f'x{I}{J}', f'x{I}{J}')] += w1 * -2 * D[I][I] * A[J][J]
        for i in range(0,n):
            Q[(f'x{i}{J}', f'x{i}{J}')] += w1 * D[I][i]^2
            Q[(f'x{I}{i}', f'x{I}{i}')] += w1 * A[i][J]^2
            for j in range(0,n):
                if i != j:
                    set_qubo_val(Q, f'x{i}{J}', f'x{j}{J}', w1 * D[I][i] * D[I][j])
                    set_qubo_val(Q, f'x{I}{i}', f'x{I}{j}', w1 * A[i][J] * A[j][J])
        for i, j in prd(range(0,n), repeat=2):
            if i != I and j != J:
                set_qubo_val(Q, f'x{i}{J}', f'x{I}{j}', -w1 * A[j][J] * D[I][i])
            if i != J and j != I:
                set_qubo_val(Q, f'x{I}{i}', f'x{j}{J}', -w1 * A[i][J] * D[I][j])

    #constraint on rows
    w2 = params['weight_row']
    for i, j in prd(range(0, n), repeat=2):
        Q[(f'x{i}{j}', f'x{i}{j}')] += w2 * -1
        for k in range(0, n):
            if k != i:
                set_qubo_val(Q, f'x{i}{j}', f'x{k}{j}', w2 * 2)
    
    #constraint on columns
    w3 = params['weight_column']
    for i, j in prd(range(0, n), repeat=2):
        Q[(f'x{i}{j}', f'x{i}{j}')] += w3 * -1
        for k in range(0, n):
            if k != j:
                set_qubo_val(Q, f'x{i}{j}', f'x{i}{k}', w3 * 2) 
    return Q